# CEWA 568: Snow Hydro
## HW#2
#### Cassie Lumbrazo

In [1]:
# import libraries we'll need
import urllib
import pandas as pd
import datetime as dt
import xarray as xr
import matplotlib.pyplot as plt
import os
%matplotlib inline

import numpy as np
from datetime import datetime
import requests

If you want to use geopandas, and contextily...then uncomment and run the two lines below

In [2]:
# !pip install --user contextily geopandas metloom

In [3]:
# import contextily as cx # this is for plotting 
# import geopandas as gpd # for location information of snotel sites

# Problem 1
## Using the observations of total precipitation and the snow pillow data (see labs above), we are going to investigate the mass balance and snow accumulation at Kettle Ponds.

### A. Plotting
* Plot a timeseries of total accumulated precipitation and total accumulated snow water equivalent from the four snow pillows. 
* Add to your plot timeseries from the two Snotel Stations located nearby. 
* Discuss how they match and how they differ, and why you think they differ.

In [4]:
ds_sos=xr.open_dataset("sos_SWE_T_P.nc")
ds_sos.head()

<xarray.Dataset>
Dimensions:   (time: 5)
Coordinates:
  * time      (time) datetime64[ns] 2022-12-01 ... 2022-12-01T04:00:00
Data variables:
    SWE_p1_c  (time) float32 ...
    SWE_p2_c  (time) float32 ...
    SWE_p3_c  (time) float32 ...
    SWE_p4_c  (time) float32 ...
    T_2m_c    (time) float32 ...

In [5]:
ds_precip = xr.open_dataset('precipitation.nc')
ds_precip.head()

<xarray.Dataset>
Dimensions:   (date: 5)
Coordinates:
  * date      (date) datetime64[ns] 2022-11-01 2022-11-02 ... 2022-11-05
Data variables:
    acc_prec  (date) float64 ...

In [6]:
sntl_ds=xr.open_dataset("sntl_ds.nc")
sntl_ds.head()

<xarray.Dataset>
Dimensions:   (Date: 5, Location: 2)
Coordinates:
  * Date      (Date) datetime64[ns] 1980-10-01 1980-10-02 ... 1980-10-05
  * Location  (Location) object '380:CO:SNTL' '737:CO:SNTL'
    WY        (Date) int64 ...
    dowy      (Date) float64 ...
Data variables:
    TAVG      (Location, Date) float64 ...
    SNWD      (Location, Date) float64 ...
    WTEQ      (Location, Date) float64 ...

In [7]:
# Function to get the day of Water Year
def date2DOWY(day_of_interest):
    day_of_interest = pd.to_datetime(day_of_interest)
    if day_of_interest >= dt.datetime(day_of_interest.year,10,1):
        dowy_start = dt.datetime(day_of_interest.year,10,1)
    else: 
        dowy_start = dt.datetime(day_of_interest.year-1,10,1)
    dowy = day_of_interest - dowy_start
    return dowy.days
# Create an empty array with the same length as the day of water year
dowy = np.zeros(len(sntl_ds.Date.values))
for i,date in enumerate(sntl_ds.Date.values):
    dowy[i] = date2DOWY(date)

# Add this to our dataset
sntl_ds = sntl_ds.assign_coords({'dowy':('Date',dowy)})

In [ ]:
fig, ax = plt.subplots()

ax.plot(sntl_ds.isel(Location=0).where(sntl_ds.WY==2023).dowy,
        sntl_ds.isel(Location=0).where(sntl_ds.WY==2023)['WTEQ'],
        label='2023 SWE')

ax.plot(sntl_ds.isel(Location=1).where(sntl_ds.WY==2023).dowy,
        sntl_ds.isel(Location=1).where(sntl_ds.WY==2023)['WTEQ'],
        label='2023 SWE')

sntl_ds.isel(Location=0).groupby(sntl_ds.dowy).median()['WTEQ'].plot(ax=ax, label='Median SWE')
ax.set_xlabel('Day of Water Year')
ax.set_title('2023 SWE and Median SWE')
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10), sharex=True)

ax.plot(ds_precip.date, ds_precip.acc_prec, label='Precipitation', color='blue', linewidth=3)

ds_sos.SWE_p1_c.plot(label='SWE_p1_c', color='indianred', linewidth=3)
ds_sos.SWE_p2_c.plot(label='SWE_p2_c', color='darkred', linewidth=3)
ds_sos.SWE_p3_c.plot(label='SWE_p3_c', color='salmon', linewidth=3)
ds_sos.SWE_p4_c.plot(label='SWE_p4_c', color='orangered', linewidth=3)


ax.plot(sntl_ds.isel(Location=0).where(sntl_ds.WY==2023).Date,
        (sntl_ds.isel(Location=0).where(sntl_ds.WY==2023)['WTEQ'])*10, # cm to mm for this dataset
        label='2023 SWE 380:CO:SNTL Butte', color='green', linewidth=3)

ax.plot(sntl_ds.isel(Location=1).where(sntl_ds.WY==2023).Date,
        (sntl_ds.isel(Location=1).where(sntl_ds.WY==2023)['WTEQ'])*10, # cm to mm for this dataset
        label='2023 SWE 737:CO:SNTL Schofield Pass', color='darkgreen', linewidth=3)


plt.legend()
plt.grid()
ax.set_ylabel("Snow pillow\nSnow Water Equivalent [mm]")
# ax.set_xlim(dt.datetime(2022,12,1), dt.datetime(2023,3,27))
plt.show()

*The snowpillows match well. There are some small differences that I am such Danny and Eli can explain, but I don't know enough about the site to explain that. The Butte SNOTEL is closer to the snow pillows and matches a lot better compared to the Schofiled Pass SNOTEL. The Schofield Pass SNOTEL is further away and ... higher in elevation I think? but I can't load the map from Lab2-1 to be able to tell for sure. The json file for plotting the map isn't on the GitHub website.*

### B. Write out the snow mass balance equation for a control volume. 
* Given the data you have plotted, what aspects of the mass balance equation do you have the most certainty about? The least certainty?

**Mass Balance Equations**

First, a list of the snow mass balance for a control volume (*everything I can think of*)

* $ \Delta SWE = Preciption_{in} + BlowingSnow_{in} - BlowingSnow_{out} - Sublimation_{out} + Deposition_{in} - Melt_{out} $


Next, simplify the equation for the changes in mass in and out

* $ \Delta SWE = \Delta Mass_{in} - \Delta Mass_{out} $

How all of the things from the above equation fit into the $\Delta Mass_{in} - \Delta Mass_{out}$

* $ \Delta SWE = Precipitation_{in} - Melt_{out} + \Delta Saltation + \Delta Sublimation  $

* $ \Delta SWE = \Delta (Accum - Ablat) + \Delta Salt + \Delta Sub $


**Mass Balance Certainty**

We are directly measuring precipitation and SWE in the field (plotting above). 
*I know we are measuring blowing snow and sublimation (latent heat of vap) in the SOS field campaign in other ways but we are not looking at that here*. 
Thus, we have more certainty in precip and SWE (and melt) here, compared to sublimation and saltation. 
There is still uncertainty in precipitation measurements from undercatch and uncertainty in SWE measurements from saltation. However, we can use the many snow pillows in one location to look at blowing snow. We could see, over a small spatial scale, if SWE is changing from one pillow to the next (when there is no precip). 

### C. What factors of the mass balance are the most similar spatially? 
* Which differ the most across different locations? 
* Is there more variability between sites further away? 
* Or are the four snow pillows at Kettle Ponds as different as the two sites just outside of the watershed?

There are two spatial scales to consider. 
1. The scale of the snow pillows all next to eachother
* At this scale, we can assume precipitation, melt, and sublimation would be the same. Thus, we can quantify $\Delta Saltation$ across the site. 
* I already see there are some differences in the snow pillows that are near eachother, so we could see how snow is transported across the site and quantify saltation. 

2. The scale of the snow pillows, compared to the SNOTEL sites
* At this scale, we cannot assume any of these variables are the same.
* We must consider differences in elevation, slope, aspect (with that, temperature, precip, etc.) and how that impacts melt

The four snow pillows at Kettle Ponds aren't as different as the two sites just outside the watershed. Of course, this is because they would have the same precip over such a small spatial scale. But, interestingly, they still do have differences. 
* Especially around a storm in the end of December (**see the below plot**) when pillow 1 has a large spike. It would be interesting to look at the wind speed over this time. 

In [ ]:
fig, ax = plt.subplots(figsize=(14,6), sharex=True)

ax.plot(ds_precip.date, ds_precip.acc_prec, label='Precipitation', color='blue', linewidth=3)

ds_sos.SWE_p1_c.plot(label='SWE_p1_c', color='indianred', linewidth=3)
ds_sos.SWE_p2_c.plot(label='SWE_p2_c', color='darkred', linewidth=3)
ds_sos.SWE_p3_c.plot(label='SWE_p3_c', color='salmon', linewidth=3)
ds_sos.SWE_p4_c.plot(label='SWE_p4_c', color='orangered', linewidth=3)


plt.legend()
plt.grid()
ax.set_ylabel("Snow pillow\nSnow Water Equivalent [mm]")
# ax.set_xlim(dt.datetime(2022,12,1), dt.datetime(2023,3,27))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14,6), sharex=True)

ax.plot(ds_precip.date, ds_precip.acc_prec, label='Precipitation', color='blue', linewidth=3)

ds_sos.SWE_p1_c.plot(label='SWE_p1_c', color='indianred', linewidth=3)
ds_sos.SWE_p2_c.plot(label='SWE_p2_c', color='darkred', linewidth=3)
ds_sos.SWE_p3_c.plot(label='SWE_p3_c', color='salmon', linewidth=3)
ds_sos.SWE_p4_c.plot(label='SWE_p4_c', color='orangered', linewidth=3)


plt.legend()
plt.grid()
ax.set_ylabel("Snow pillow\nSnow Water Equivalent [mm]")
ax.set_xlim(dt.datetime(2022,12,17), dt.datetime(2022,12,30))
ax.set_ylim(0,200)
plt.show()

### D. Prediction NCAR's EOL team is committed to removing the towers at Kettle Ponds on 1 June 2023. 
To do that, they need to drive down a road (shown in the map above) that is currently snowed in. The road will open when the snow melts and disappears. This typically occurs in mid to late May. Using the long term records at the SNOTEL sites, and any other information you may have, to make a prediction about whether the snow will be gone (and hence the road clear) by 1 June 2023. (Note that we will assess everyone's accuracy at the end of the quarter!)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5), sharex=True)

WYS=[2015,2016,2017,2018,2019,2020,2021,2022]

ax.plot(sntl_ds.isel(Location=0).where(sntl_ds.WY.isin(WYS)).Date,
        (sntl_ds.isel(Location=0).where(sntl_ds.WY.isin(WYS))['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax.plot(sntl_ds.isel(Location=1).where(sntl_ds.WY.isin(WYS)).Date,
        (sntl_ds.isel(Location=1).where(sntl_ds.WY.isin(WYS))['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')

plt.legend()
ax.set_ylabel("Snow pillow,\nSnow Water Equivalent [mm]")
# ax.set_xlim(dt.datetime(2022,12,1), dt.datetime(2023,3,27))
plt.show()

In [ ]:
ds_stl = sntl_ds.copy(deep=True)

In [ ]:
fig, ax = plt.subplots(9,1,figsize=(12,20), sharey=True)

WYS=[2015,2016,2017,2018,2019,2020,2021,2022]

#____________________________________________________________________________________________
ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2015).Date,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2015)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax[0].plot(ds_stl.isel(Location=1).where(ds_stl.WY==2015).Date,
        (ds_stl.isel(Location=1).where(ds_stl.WY==2015)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')

#____________________________________________________________________________________________
ax[1].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2016).Date,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2016)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax[1].plot(ds_stl.isel(Location=1).where(ds_stl.WY==2016).Date,
        (ds_stl.isel(Location=1).where(ds_stl.WY==2016)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')

#____________________________________________________________________________________________
ax[2].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2017).Date,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2017)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax[2].plot(ds_stl.isel(Location=1).where(ds_stl.WY==2017).Date,
        (ds_stl.isel(Location=1).where(ds_stl.WY==2017)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')

#____________________________________________________________________________________________
ax[3].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2018).Date,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2018)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax[3].plot(ds_stl.isel(Location=1).where(ds_stl.WY==2018).Date,
        (ds_stl.isel(Location=1).where(ds_stl.WY==2018)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')
#____________________________________________________________________________________________
ax[4].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2019).Date,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2019)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax[4].plot(ds_stl.isel(Location=1).where(ds_stl.WY==2019).Date,
        (ds_stl.isel(Location=1).where(ds_stl.WY==2019)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')

#____________________________________________________________________________________________
ax[5].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2020).Date,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2020)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax[5].plot(ds_stl.isel(Location=1).where(ds_stl.WY==2020).Date,
        (ds_stl.isel(Location=1).where(ds_stl.WY==2020)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')

#____________________________________________________________________________________________
ax[6].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2021).Date,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2021)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax[6].plot(ds_stl.isel(Location=1).where(ds_stl.WY==2021).Date,
        (ds_stl.isel(Location=1).where(ds_stl.WY==2021)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')

#____________________________________________________________________________________________
ax[7].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2022).Date,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2022)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax[7].plot(ds_stl.isel(Location=1).where(ds_stl.WY==2022).Date,
        (ds_stl.isel(Location=1).where(ds_stl.WY==2022)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')

#____________________________________________________________________________________________
ax[8].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2023).Date,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2023)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

ax[8].plot(ds_stl.isel(Location=1).where(ds_stl.WY==2023).Date,
        (ds_stl.isel(Location=1).where(ds_stl.WY==2023)['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 737:CO:SNTL Schofield Pass')

ax[0].legend()
ax[4].set_ylabel("Snow pillow, Snow Water Equivalent [mm]")
# ax.set_xlim(dt.datetime(2022,12,1), dt.datetime(2023,3,27))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5), sharex=True)

WYS=[2015,2016,2017,2018,2019,2020,2021,2022]

ax.plot(ds_stl.isel(Location=0).where(ds_stl.WY.isin(WYS)).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY.isin(WYS))['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')


plt.legend()
ax.set_ylabel("Snow pillow,\nSnow Water Equivalent [mm]")
# ax.set_xlim(dt.datetime(2022,12,1), dt.datetime(2023,3,27))
plt.show()

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(12,5), sharex=True)

WYS=[2015,2016,2017,2018,2019,2020,2021,2022]

ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY.isin(WYS)).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY.isin(WYS))['WTEQ'])*10, # cm to mm for this dataset
        label='SWE 380:CO:SNTL Butte')

WY=2023
ax[1].plot(ds_stl.isel(Location=0).where(ds_stl.WY.isin(WY)).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY.isin(WY))['WTEQ'])*10, # cm to mm for this dataset
        label='2023 SWE 380:CO:SNTL Butte')


plt.legend()
ax[0].set_ylabel("Snow pillow,\nSnow Water Equivalent [mm]")
# ax.set_xlim(dt.datetime(2022,12,1), dt.datetime(2023,3,27))
plt.show()

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(12,10), sharex=True)

year=2015;yearr='2015'
ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY==year).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==year)['WTEQ'])*10, # cm to mm for this dataset
        label=yearr)

year=2016;yearr='2016'
ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY==year).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==year)['WTEQ'])*10, # cm to mm for this dataset
        label=yearr)

year=2017;yearr='2017'
ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY==year).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==year)['WTEQ'])*10, # cm to mm for this dataset
        label=yearr)

year=2018;yearr='2018'
ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY==year).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==year)['WTEQ'])*10, # cm to mm for this dataset
        label=yearr)

year=2019;yearr='2019'
ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY==year).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==year)['WTEQ'])*10, # cm to mm for this dataset
        label=yearr)

year=2020;yearr='2020'
ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY==year).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==year)['WTEQ'])*10, # cm to mm for this dataset
        label=yearr)

year=2021;yearr='2021'
ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY==year).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==year)['WTEQ'])*10, # cm to mm for this dataset
        label=yearr)

ax[0].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2023).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2023)['WTEQ'])*10, # cm to mm for this dataset
        label='2023', color='k', linewidth=3)

ax[0].legend()

year=2017;yearr='2017'
ax[1].plot(ds_stl.isel(Location=0).where(ds_stl.WY==year).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==year)['WTEQ'])*10, # cm to mm for this dataset
        label=yearr)

ax[1].plot(ds_stl.isel(Location=0).where(ds_stl.WY==2023).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2023)['WTEQ'])*10, # cm to mm for this dataset
        label='2023', color='k', linewidth=3)
ax[1].legend()

ax[0].set_ylabel("Snow pillow,\nSnow Water Equivalent [mm]")
ax[1].set_ylabel("Snow pillow,\nSnow Water Equivalent [mm]")
# ax.set_xlim(dt.datetime(2022,12,1), dt.datetime(2023,3,27))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,4), sharex=True)

year=2017;yearr='2017'
ax.plot(ds_stl.isel(Location=0).where(ds_stl.WY==year).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==year)['WTEQ'])*10, # cm to mm for this dataset
        label=yearr, linewidth=3)

ax.plot(ds_stl.isel(Location=0).where(ds_stl.WY==2023).dowy,
        (ds_stl.isel(Location=0).where(ds_stl.WY==2023)['WTEQ'])*10, # cm to mm for this dataset
        label='2023', color='k', linewidth=3)
ax.legend()

ax.set_ylabel("Snow pillow,\nSnow Water Equivalent [mm]")
ax.set_ylabel("Snow pillow,\nSnow Water Equivalent [mm]")
ax.set_xlim(160, 240)
plt.grid()
plt.show()

The current SWE for WY2023 is very close to WY2017 (only ~2 days different right now). If we go by this only, then I think the snow melt out date would be between May 25-May 27. 
Of course, this is not considering all the other factors that contribute to snowmelt timing... but I am not going to build a bigger model for this now and instead, go with **May 27th** as my guess. 

# Problem 2: Course Project Selection
Write a brief paragraph (3-9 sentences) describing your term project. 
* Include the name of your partner (or specify you will be doing the project alone)
* The concept you will communicate or teach
* The tools you will use. 

If you have questions about the project, now is the time to talk to the instructor.

### Idea 1. 
#### ArcGIS model for managers.
**Description:** ArcGIS model for managers. I could create a video of myself to post on youtube, explaining how my model works and the steps I take to turn a LiDAR raster into a map of distance from canopy, and compare with snowdepth, slope, aspect, etc. 
* interested to produce a 3D model of our sites on CER using the NCALM lidar data.... 

Audience: forest managers or researchers who want to use my methods to evaluate their forest-snow relationships

Partner: None

$OR$

### Idea 2. 
#### Forcing data *how to* for students
**Description:**  Create a juptyer notebook that follows Jessica's snow hydrology class lecture on forcing data. The goal would be to produce a jupyter notebook that I can send to students next time someone asks me "how do I create a SUMMA forcing file?". 
* walking through forcing data basics 
* include analysis of SW radiation, making sure that $SW_{in} < SW_{POT}$
* include tricks for updating canopy height in other model files (using netcdfs, xarray)
* include the metadata and code needed to convert your clean forcing data into a tidy netcdf forcing file for SUMMA

Audience: first year graduate students trying to run SUMMA

Partner: Ross (or None)
* if Ross joins, he would collab with me and write the end of his script to produce a SnowModel forcing file with required metdata